Простой пример: Уведомления с дополнительными опциями
Представим систему уведомлений. У нас есть базовый способ отправки (например, email), и мы хотим динамически добавлять другие каналы (SMS, Slack) или пометки (срочность), не меняя исходный класс и не создавая множество подклассов для каждой комбинации.

In [1]:
from abc import ABC, abstractmethod

# 1. Component Interface (Интерфейс Компонента)
class Notifier(ABC):
    """Интерфейс для отправки уведомлений."""
    @abstractmethod
    def send(self, message: str) -> None:
        pass

# 2. Concrete Component (Конкретный Компонент - базовая отправка)
class EmailNotifier(Notifier):
    """Базовая реализация: отправка по email."""
    def send(self, message: str) -> None:
        print(f"Sending email with message: '{message}'")

# 3. Base Decorator (Базовый Декоратор - необязателен, но удобен)
class BaseNotifierDecorator(Notifier):
    """Абстрактный базовый класс для декораторов."""
    _wrapped: Notifier = None # Ссылка на оборачиваемый объект

    def __init__(self, notifier: Notifier):
        self._wrapped = notifier

    @abstractmethod
    def send(self, message: str) -> None:
        """Делегирует работу обернутому компоненту."""
        self._wrapped.send(message)

# 4. Concrete Decorators (Конкретные Декораторы)
class SMSDecorator(BaseNotifierDecorator):
    """Декоратор, добавляющий отправку SMS."""
    def send(self, message: str) -> None:
        # Сначала вызываем базовый метод (например, email)
        super().send(message)
        # Затем добавляем свою функциональность
        print(f"Sending SMS with message: '{message}'")

class SlackDecorator(BaseNotifierDecorator):
    """Декоратор, добавляющий отправку в Slack."""
    def send(self, message: str) -> None:
        super().send(message)
        print(f"Sending Slack message: '{message}'")

class UrgentDecorator(BaseNotifierDecorator):
    """Декоратор, добавляющий пометку о срочности."""
    def send(self, message: str) -> None:
        urgent_message = f"[URGENT] {message}"
        super().send(urgent_message) # Отправляем измененное сообщение дальше по цепочке

# 5. Client Code (Клиентский Код)
if __name__ == "__main__":
    # Базовый уведомитель
    email_notifier = EmailNotifier()
    print("--- Sending with basic EmailNotifier ---")
    email_notifier.send("Hello there!")

    print("\n--- Decorating with SMS ---")
    # Оборачиваем email в SMS декоратор
    sms_notifier = SMSDecorator(email_notifier)
    sms_notifier.send("Package delivered.")

    print("\n--- Decorating with Slack and SMS ---")
    # Можно вкладывать декораторы: email -> sms -> slack
    slack_sms_notifier = SlackDecorator(SMSDecorator(email_notifier))
    slack_sms_notifier.send("Meeting starts in 5 minutes.")

    print("\n--- Decorating with Urgency, Slack, and SMS ---")
    # email -> sms -> slack -> urgent
    # Порядок важен! Urgent будет применен последним, изменив сообщение
    # для всех предыдущих шагов.
    urgent_slack_sms_notifier = UrgentDecorator(
                                    SlackDecorator(
                                        SMSDecorator(email_notifier)
                                    )
                                 )
    urgent_slack_sms_notifier.send("System critical error!")

    print("\n--- Decorating with Urgency applied first ---")
    # email -> urgent -> sms -> slack
    # Порядок изменен: Urgent применится к сообщению только для email
    # SMS и Slack получат уже измененное сообщение.
    sms_slack_urgent_notifier = SlackDecorator(
                                    SMSDecorator(
                                        UrgentDecorator(email_notifier)
                                    )
                                )
    sms_slack_urgent_notifier.send("Server backup complete.")


# Вывод:
# --- Sending with basic EmailNotifier ---
# Sending email with message: 'Hello there!'
#
# --- Decorating with SMS ---
# Sending email with message: 'Package delivered.'
# Sending SMS with message: 'Package delivered.'
#
# --- Decorating with Slack and SMS ---
# Sending email with message: 'Meeting starts in 5 minutes.'
# Sending SMS with message: 'Meeting starts in 5 minutes.'
# Sending Slack message: 'Meeting starts in 5 minutes.'
#
# --- Decorating with Urgency, Slack, and SMS ---
# Sending email with message: '[URGENT] System critical error!'
# Sending SMS with message: '[URGENT] System critical error!'
# Sending Slack message: '[URGENT] System critical error!'
#
# --- Decorating with Urgency applied first ---
# Sending email with message: '[URGENT] Server backup complete.'
# Sending SMS with message: '[URGENT] Server backup complete.'
# Sending Slack message: '[URGENT] Server backup complete.'

--- Sending with basic EmailNotifier ---
Sending email with message: 'Hello there!'

--- Decorating with SMS ---
Sending email with message: 'Package delivered.'
Sending SMS with message: 'Package delivered.'

--- Decorating with Slack and SMS ---
Sending email with message: 'Meeting starts in 5 minutes.'
Sending SMS with message: 'Meeting starts in 5 minutes.'
Sending Slack message: 'Meeting starts in 5 minutes.'

--- Decorating with Urgency, Slack, and SMS ---
Sending email with message: '[URGENT] System critical error!'
Sending SMS with message: '[URGENT] System critical error!'
Sending Slack message: '[URGENT] System critical error!'

--- Decorating with Urgency applied first ---
Sending email with message: '[URGENT] Server backup complete.'
Sending SMS with message: 'Server backup complete.'
Sending Slack message: 'Server backup complete.'


Сложный пример: Получение данных с кэшированием и логированием
Представим компонент, который получает данные по ключу (например, из сети). Мы хотим добавить кэширование, чтобы избежать повторных запросов, и логирование для отладки.

In [2]:
import time
import json
from abc import ABC, abstractmethod
from typing import Any, Dict

# 1. Component Interface (Интерфейс Компонента)
class DataSource(ABC):
    """Интерфейс для получения данных."""
    @abstractmethod
    def get_data(self, key: str) -> Any:
        pass

# 2. Concrete Component (Конкретный Компонент - получение из 'сети')
class WebDataSource(DataSource):
    """Имитирует получение данных из веб-источника с задержкой."""
    def get_data(self, key: str) -> Any:
        print(f"WebDataSource: Fetching data for key '{key}' from web...")
        # Имитация сетевой задержки
        time.sleep(1.5)
        # Имитация ответа (может быть JSON строка)
        if key == "user:123":
            return '{"name": "Alice", "id": 123, "status": "active"}'
        elif key == "products":
            return '[{"sku": "P1", "price": 100}, {"sku": "P2", "price": 50}]'
        else:
            return None # Или '{}' для JSON парсера

# 3. Base Decorator (Базовый Декоратор)
class DataSourceDecorator(DataSource):
    _wrapped: DataSource

    def __init__(self, source: DataSource):
        self._wrapped = source

    @abstractmethod
    def get_data(self, key: str) -> Any:
        return self._wrapped.get_data(key)

# 4. Concrete Decorators (Конкретные Декораторы)
class CachingDecorator(DataSourceDecorator):
    """Декоратор, добавляющий кэширование."""
    _cache: Dict[str, Any] = {}
    _cache_ttl: Dict[str, float] = {} # Время жизни кэша (простой пример)
    _ttl_seconds: float = 5.0 # Кэш живет 5 секунд

    def get_data(self, key: str) -> Any:
        current_time = time.time()
        # Проверяем, есть ли ключ в кэше и не истек ли он
        if key in self._cache and (current_time - self._cache_ttl.get(key, 0)) < self._ttl_seconds:
            print(f"CacheDecorator: Returning cached data for key '{key}'")
            return self._cache[key]
        else:
            # Данных нет или кэш устарел, получаем свежие
            print(f"CacheDecorator: Cache miss or expired for key '{key}'. Fetching fresh data.")
            data = super().get_data(key) # Вызываем метод обернутого компонента
            # Кэшируем результат
            self._cache[key] = data
            self._cache_ttl[key] = current_time
            print(f"CacheDecorator: Data for key '{key}' cached.")
            return data

class LoggingDecorator(DataSourceDecorator):
    """Декоратор, добавляющий логирование запросов."""
    def get_data(self, key: str) -> Any:
        print(f"LoggingDecorator: Requesting data for key '{key}'...")
        start_time = time.time()
        try:
            result = super().get_data(key) # Вызываем метод обернутого компонента
            end_time = time.time()
            print(f"LoggingDecorator: Request for key '{key}' completed in {end_time - start_time:.2f}s. Result type: {type(result)}")
            return result
        except Exception as e:
            end_time = time.time()
            print(f"LoggingDecorator: Request for key '{key}' failed in {end_time - start_time:.2f}s with error: {e}")
            raise # Перебрасываем исключение дальше

class JsonParsingDecorator(DataSourceDecorator):
    """Декоратор, который пытается распарсить результат как JSON."""
    def get_data(self, key: str) -> Any:
        raw_data = super().get_data(key) # Получаем данные от предыдущего слоя
        if isinstance(raw_data, str):
            try:
                print(f"JsonParsingDecorator: Attempting to parse JSON for key '{key}'...")
                parsed_data = json.loads(raw_data)
                print(f"JsonParsingDecorator: JSON parsed successfully for key '{key}'.")
                return parsed_data
            except json.JSONDecodeError:
                print(f"JsonParsingDecorator: Failed to parse JSON for key '{key}'. Returning raw data.")
                return raw_data # Возвращаем как есть, если не JSON
        else:
            # Если данные не строка, просто возвращаем их
            print(f"JsonParsingDecorator: Data for key '{key}' is not a string, skipping JSON parsing.")
            return raw_data

# 5. Client Code
if __name__ == "__main__":
    # Создаем базовый источник данных
    source = WebDataSource()

    print("--- Using WebDataSource directly ---")
    data1 = source.get_data("user:123")
    print(f"Received data type: {type(data1)}, value: {data1}")
    data2 = source.get_data("user:123") # Снова долгий запрос
    print(f"Received data type: {type(data2)}, value: {data2}")

    print("\n--- Using Decorated DataSource (Log -> Cache -> JSON -> Web) ---")
    # Оборачиваем источник декораторами
    # Порядок: Логируем -> Проверяем кэш -> Если нет, парсим JSON -> Если нет, идем в Web
    decorated_source = LoggingDecorator(
                            CachingDecorator(
                                JsonParsingDecorator(
                                    WebDataSource() # Новый экземпляр для чистоты эксперимента
                                )
                            )
                        )

    print("\n[Request 1: user:123]")
    user_data = decorated_source.get_data("user:123")
    print(f"Client received data type: {type(user_data)}, value: {user_data}")

    print("\n[Request 2: user:123 (should be cached)]")
    user_data_cached = decorated_source.get_data("user:123")
    print(f"Client received data type: {type(user_data_cached)}, value: {user_data_cached}")

    print("\n[Request 3: products]")
    products_data = decorated_source.get_data("products")
    print(f"Client received data type: {type(products_data)}, value: {products_data}")

    print("\n[Request 4: user:123 (cache should expire after 5 seconds)]")
    print("Waiting for cache to expire (6 seconds)...")
    time.sleep(6)
    user_data_expired = decorated_source.get_data("user:123")
    print(f"Client received data type: {type(user_data_expired)}, value: {user_data_expired}")


# Примерный вывод (время выполнения будет отличаться):
# --- Using WebDataSource directly ---
# WebDataSource: Fetching data for key 'user:123' from web...
# Received data type: <class 'str'>, value: {"name": "Alice", "id": 123, "status": "active"}
# WebDataSource: Fetching data for key 'user:123' from web...
# Received data type: <class 'str'>, value: {"name": "Alice", "id": 123, "status": "active"}
#
# --- Using Decorated DataSource (Log -> Cache -> JSON -> Web) ---
#
# [Request 1: user:123]
# LoggingDecorator: Requesting data for key 'user:123'...
# CacheDecorator: Cache miss or expired for key 'user:123'. Fetching fresh data.
# JsonParsingDecorator: Attempting to parse JSON for key 'user:123'...
# WebDataSource: Fetching data for key 'user:123' from web...
# JsonParsingDecorator: JSON parsed successfully for key 'user:123'.
# CacheDecorator: Data for key 'user:123' cached.
# LoggingDecorator: Request for key 'user:123' completed in 1.50s. Result type: <class 'dict'>
# Client received data type: <class 'dict'>, value: {'name': 'Alice', 'id': 123, 'status': 'active'}
#
# [Request 2: user:123 (should be cached)]
# LoggingDecorator: Requesting data for key 'user:123'...
# CacheDecorator: Returning cached data for key 'user:123'
# LoggingDecorator: Request for key 'user:123' completed in 0.00s. Result type: <class 'dict'>
# Client received data type: <class 'dict'>, value: {'name': 'Alice', 'id': 123, 'status': 'active'}
#
# [Request 3: products]
# LoggingDecorator: Requesting data for key 'products'...
# CacheDecorator: Cache miss or expired for key 'products'. Fetching fresh data.
# JsonParsingDecorator: Attempting to parse JSON for key 'products'...
# WebDataSource: Fetching data for key 'products' from web...
# JsonParsingDecorator: JSON parsed successfully for key 'products'.
# CacheDecorator: Data for key 'products' cached.
# LoggingDecorator: Request for key 'products' completed in 1.50s. Result type: <class 'list'>
# Client received data type: <class 'list'>, value: [{'sku': 'P1', 'price': 100}, {'sku': 'P2', 'price': 50}]
#
# [Request 4: user:123 (cache should expire after 5 seconds)]
# Waiting for cache to expire (6 seconds)...
# LoggingDecorator: Requesting data for key 'user:123'...
# CacheDecorator: Cache miss or expired for key 'user:123'. Fetching fresh data.
# JsonParsingDecorator: Attempting to parse JSON for key 'user:123'...
# WebDataSource: Fetching data for key 'user:123' from web...
# JsonParsingDecorator: JSON parsed successfully for key 'user:123'.
# CacheDecorator: Data for key 'user:123' cached.
# LoggingDecorator: Request for key 'user:123' completed in 1.50s. Result type: <class 'dict'>
# Client received data type: <class 'dict'>, value: {'name': 'Alice', 'id': 123, 'status': 'active'}

--- Using WebDataSource directly ---
WebDataSource: Fetching data for key 'user:123' from web...
Received data type: <class 'str'>, value: {"name": "Alice", "id": 123, "status": "active"}
WebDataSource: Fetching data for key 'user:123' from web...
Received data type: <class 'str'>, value: {"name": "Alice", "id": 123, "status": "active"}

--- Using Decorated DataSource (Log -> Cache -> JSON -> Web) ---

[Request 1: user:123]
LoggingDecorator: Requesting data for key 'user:123'...
CacheDecorator: Cache miss or expired for key 'user:123'. Fetching fresh data.
WebDataSource: Fetching data for key 'user:123' from web...
JsonParsingDecorator: Attempting to parse JSON for key 'user:123'...
JsonParsingDecorator: JSON parsed successfully for key 'user:123'.
CacheDecorator: Data for key 'user:123' cached.
LoggingDecorator: Request for key 'user:123' completed in 1.50s. Result type: <class 'dict'>
Client received data type: <class 'dict'>, value: {'name': 'Alice', 'id': 123, 'status': 'active'}

